## Asserting Facts into the Engine

You can send Assertions, also known as Facts, into the engine for processing. The engine will match the incoming Assertions against its KB (knowledge base) and trigger any matching rules. Once those rules trigger, they can cause additional rules to trigger, and so on, until no additional assertions are generated.

This tutorial will walk you through the different ways of controlling the assertion process.

Run the following code to get the engine started.

In [1]:
# import the thoughts rules engine
import os, sys
sys.path.insert(1, os.path.abspath('..\\..\\..'))
from thoughts.rules_engine import RulesEngine
from pprint import pprint

engine = RulesEngine()

## A Simple Example - Default Behavior for Assertions

By default when a new fact is asserted, the engine will run continually until no additional assertions are generated. 

It will also return the final conclusions generated by the engine, hiding any interim assertions that were gerneated during processing.

Let's run a trivial example to see both of these behaviors in action.

In [2]:
rules = [
    {"#when": "A", "#then": "B"},
    {"#when": "B", "#then": "C"},
    {"#when": "C", "#then": "D"},
]

engine.load_rules_from_list(rules)

conclusions = engine.process("A")
pprint(conclusions)

['D']


In the example above, asserting 'A' into the engine causes 'B' to fire. 'B' causes 'C' to fire, and finally 'C' causes 'D' to fire. The engine processed all assertions until no additional assertions were triggered. 

Notice that the output only produces the final assertion. You do not see the interim assertions generated. This is the default behavior when you assert a fact into the engine. Below we'll see how to control this.

## Viewing the Assertion Tree

Now let's run the same assertion again but this time we'll return the entire set of conclusions, including the interim assertions generated. You can do this by setting the extract_conclusions parameter to False, causing the engine to return the full assertion tree.

Notice how this representation shows how A led to B, which led to C, which led to D. Each conclusion is tracked as sub-conclusion of the parent conclusion which triggered it.

In [3]:
conclusions = engine.process("A", extract_conclusions=False)
pprint(conclusions)

[{'#assert': 'A',
  '#conclusions': [{'#assert': 'B',
                    '#conclusions': [{'#assert': 'C',
                                      '#conclusions': [{'#assert': 'D',
                                                        '#conclusions': []}]}]}],
  '#seq-end': 1,
  '#seq-start': 0}]


## Processing Assertions One at a Time

You can also tell the engine to process an assertion and to halt as soon as the next conclusions are triggered, but not to automatically trigger additonal rules based on those conclusions. This allows you to step through interim assertions one at a time for fine control or debugging of the assertion process.

In [4]:
conclusions = engine.process_single("A")
pprint(conclusions)

[{'#assert': 'A',
  '#conclusions': [{'#assert': 'B'}],
  '#seq-end': 1,
  '#seq-start': 0}]


Here 'A' was asserted, which triggered 'B'. At this point the engine stopped. Now we can sent the assertion tree result back into the engine to continue processing.

Key concept:
*To continue processing, you can send the current assertion tree back into the engine. It will pick back up where it left off in processing the next assertion which has not already been processed.*

Let's try it now:

In [5]:
conclusions = engine.process_single(conclusions)
pprint(conclusions)

[{'#assert': 'A',
  '#conclusions': [{'#assert': 'B', '#conclusions': [{'#assert': 'C'}]}],
  '#seq-end': 1,
  '#seq-start': 0}]


Send the resulting conclusionss back into the engine to process the next assertion, 'C':

In [6]:
conclusions = engine.process_single(conclusions)
pprint(conclusions)

[{'#assert': 'A',
  '#conclusions': [{'#assert': 'B',
                    '#conclusions': [{'#assert': 'C',
                                      '#conclusions': [{'#assert': 'D'}]}]}],
  '#seq-end': 1,
  '#seq-start': 0}]


Finally, to assert the 'D':

In [7]:
conclusions = engine.process_single(conclusions)
pprint(conclusions)

[{'#assert': 'A',
  '#conclusions': [{'#assert': 'B',
                    '#conclusions': [{'#assert': 'C',
                                      '#conclusions': [{'#assert': 'D',
                                                        '#conclusions': []}]}]}],
  '#seq-end': 1,
  '#seq-start': 0}]


Note that the #conclusions exist on 'D', but it's an empty list.

Asserting again, returns the same final tree.

In [8]:
conclusions = engine.process_single(conclusions)
pprint(conclusions)

[{'#assert': 'A',
  '#conclusions': [{'#assert': 'B',
                    '#conclusions': [{'#assert': 'C',
                                      '#conclusions': [{'#assert': 'D',
                                                        '#conclusions': []}]}]}],
  '#seq-end': 1,
  '#seq-start': 0}]


## Summary

Those are the basics of controlling the assertion process. In the next tutorial, we walk through sequence-based rules